In [1]:
import TouristDemoHelper as tourist_demo
model = tourist_demo.load_model("landmark")

OLED screen is not connected!
OLED screen is not connected!


Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 56, got 52
  return f(*args, **kwds)


model is loaded 🥂


In [ ]:
tourist_demo.calibrate_motors()
tourist_demo.drive_to_landmark("china", model)

'right'

'up 6.30%  left 0.70%  right 66.68%  nyc 0.00%  china 0.00%  eiffel 26.31%  '